In [1]:
import numpy as np
import pandas as pd 
import sklearn
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
# from tqdm import tqdm_notebook
from tqdm.notebook import tqdm

import catboost as cb
import xgboost as xgb

# Train/test split

For a sake of the programming assignment, let's artificially split the data into train and test. We will treat last month data as the test set.

In [ ]:
all_data = pd.read_parquet('data/all_data.parquet')
# print('Columns:', all_data.columns)
all_data.info()
all_data

In [ ]:
list(all_data.columns)

In [ ]:
all_data = all_data[[
     'shop_id',
     'item_id',
     'date_block_num',
     'target',
     'target_shop',
     'target_item',
     'target_mean',
     'target_shop_mean',
     'target_item_mean',
#      'target_item_shop_mean',
     'target_lag_1',
     'target_item_lag_1',
     'target_item_mean_lag_1',
     #'target_item_shop_mean_lag_1',
     'target_mean_lag_1',
     'target_shop_lag_1',
     'target_shop_mean_lag_1',
     'target_lag_2',
     'target_item_lag_2',
     'target_item_mean_lag_2',
     #'target_item_shop_mean_lag_2',
     'target_mean_lag_2',
     'target_shop_lag_2',
     'target_shop_mean_lag_2',
     'target_lag_3',
     'target_item_lag_3',
     'target_item_mean_lag_3',
     #'target_item_shop_mean_lag_3',
     'target_mean_lag_3',
     'target_shop_lag_3',
     'target_shop_mean_lag_3',
     'target_lag_4',
     'target_item_lag_4',
     'target_item_mean_lag_4',
     #'target_item_shop_mean_lag_4',
     'target_mean_lag_4',
     'target_shop_lag_4',
     'target_shop_mean_lag_4',
     'target_lag_5',
     'target_item_lag_5',
     'target_item_mean_lag_5',
     #'target_item_shop_mean_lag_5',
     'target_mean_lag_5',
     'target_shop_lag_5',
     'target_shop_mean_lag_5',
     'target_lag_12',
     'target_item_lag_12',
     'target_item_mean_lag_12',
#      'target_item_shop_mean_lag_12',
     'target_mean_lag_12',
     'target_shop_lag_12',
     'target_shop_mean_lag_12',
     'item_category_id',
     'city_code',
     'type_code',
     'subtype_code',
     'month_num',
     'item_first_sale',
     'item_shop_first_sale',
     'days'
]]

In [ ]:
# List of all lagged features

# We will drop these at fitting stage
to_drop_cols = ['target_shop_mean', 'target_shop', 
                'target_item', 'target', 'target_item_mean', 'target_mean', 'date_block_num'] #, 'target_item_shop_mean']

# Categorical features
cat_features = ['shop_id', 'item_id', 'item_category_id', 'city_code', 
                'type_code', 'subtype_code']

In [ ]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()-1
print('Test `date_block_num` is %d' % last_block)

In [ ]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train =  all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =   all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)
X_target = all_data.loc[dates == dates.max()].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

In [ ]:
X_train

# First level models 

You need to implement a basic stacking scheme. We have a time component here, so we will use ***scheme f)*** from the reading material. Recall, that we always use first level models to build two datasets: test meta-features and 2-nd level train-metafetures. Let's see how we get test meta-features first. 

### Test meta-features

Firts, we will run *linear regression* on numeric columns and get predictions for the last month.

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train.values, y_train)
pred_lr = model_lr.predict(X_test.values)

print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))
print('Test RMSE linreg is %f' % np.sqrt(mean_squared_error(y_test, pred_lr)))

In [ ]:
model_lor = LogisticRegression(random_state=17, n_jobs=4, solver='saga', verbose=1)
model_lor.fit(X_train.values, y_train)
pred_lor = model_lor.predict(X_test.values)

print('Test R-squared for LogReg is %f' % r2_score(y_test, pred_lor))
print('Test RMSE LogReg is %f' % np.sqrt(mean_squared_error(y_test, pred_lor)))

And the we run *LightGBM*.

In [ ]:
%%time
# LightGGM
model_lgb = lgb.LGBMRegressor(
    n_estimators=1000,
    n_jobs=-1,
    objective='mse',
    random_state=17
)
model_lgb.fit(X_train, y_train, 
              eval_set=(X_test, y_test), 
              early_stopping_rounds=100,
              verbose=100)
pred_lgb = model_lgb.predict(X_test)

print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))
print('Test RMSE LightGBM is %f' % np.sqrt(mean_squared_error(y_test, pred_lgb)))

In [ ]:
%%time
# Catboost

model_cbr = cb.CatBoostRegressor(loss_function='RMSE',
                                 cat_features=cat_features,
                                 iterations=1000,
                                task_type='CPU')
model_cbr.fit(X_train, y_train, 
              eval_set=(X_test, y_test), 
              verbose=20, 
              early_stopping_rounds=50,
              plot=True)
pred_cb = model_cbr.predict(X_test)

print('Test R-squared for CatBoost is %f' % r2_score(y_test, pred_cb))
print('Test RMSE CatBoost is %f' % np.sqrt(mean_squared_error(y_test, pred_cb)))

In [ ]:
f_importance = model_cbr.get_feature_importance(prettified=True)
f_importance.sort_values(by='Importances', ascending=True).set_index('Feature Id').plot(
    kind='barh', figsize=(10,10), legend=False, grid=True, title="Feature importances");

In [ ]:
# SVM
# model_svr = SVR()
# model_svr.fit(X_train.values, y_train)
# pred_svr = model_svr.predict(X_test.values, verbose=True)

# print('Test R-squared for SVR is %f' % r2_score(y_test, pred_svr))
# print('Test RMSE SVR is %f' % np.sqrt(mean_squared_error(y_test, pred_svr)))

In [ ]:
%%time
# XGBM
model_xgb = xgb.XGBRegressor(n_estimators=500, n_jobs=4, random_state=17)
model_xgb.fit(X_train, y_train, 
              eval_set=[(X_test, y_test)], 
              eval_metric="rmse", 
              verbose=10, 
              early_stopping_rounds = 50)
pred_xgb = model_xgb.predict(X_test)

print('Test R-squared for XGBoost is %f' % r2_score(y_test, pred_xgb))
print('Test RMSE XGBoost is %f' % np.sqrt(mean_squared_error(y_test, pred_xgb)))

In [ ]:
# Plot XGBoost features importance
from xgboost import plot_importance
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1,1,figsize=(10, 10))
plot_importance(booster=model_xgb, ax=ax)

Finally, concatenate test predictions to get test meta-features.

In [ ]:
%%time
# Make predictions for test data
train_lr  = model_lr.predict(X_train)
train_lgb = model_lgb.predict(X_train)
train_cbr = model_cbr.predict(X_train)
train_xgb = model_xgb.predict(X_train)

X_train_level2 = np.c_[train_lr, train_lgb, train_cbr, train_xgb] 
# X_test_level2.tofile('X_test_level2')

In [ ]:
X_train_level2, X_train_level2.shape

In [ ]:
%%time
# Make predictions for test data
test_lr  = model_lr.predict(X_test)
test_lgb = model_lgb.predict(X_test)
test_cbr = model_cbr.predict(X_test)
test_xgb = model_xgb.predict(X_test)

X_test_level2 = np.c_[test_lr, test_lgb, test_cbr, test_xgb]

In [ ]:
X_test_level2, X_test_level2.shape

In [ ]:
%%time
# Make prediction for target data
target_lr  = model_lr.predict(X_target)
target_lgb = model_lgb.predict(X_target)
target_cbr  = model_cbr.predict(X_target)
target_xgb = model_xgb.predict(X_target)

X_target_level2 = np.c_[target_lr, target_lgb, target_cbr, target_xgb]

In [ ]:
X_target_level2, X_target_level2.shape

In [ ]:
l2_model_lr = LinearRegression()
l2_model_lr.fit(X_train_level2, y_train)
test_l2_lr = l2_model_lr.predict(X_test_level2)

print(f'Test RMSE for LinearRegression L2 is {np.sqrt(mean_squared_error(y_test, test_l2_lr))}')

In [ ]:
l2_model_lor = LinearRegression()
l2_model_lor.fit(X_train_level2, y_train)
test_l2_lor = l2_model_lor.predict(X_test_level2)

print(f'Test RMSE for LogisticRegression L2 is {np.sqrt(mean_squared_error(y_test, test_l2_lor))}')

In [ ]:
l2_model_cb = cb.CatBoostRegressor(loss_function='RMSE',
                               task_type='CPU')
l2_model_cb.fit(X_train_level2, y=y_train,
             eval_set=(X_test_level2, y_test),
             early_stopping_rounds=50,
             verbose=20,
             plot=True)
test_l2_cbr = l2_model_cb.predict(X_test_level2)

print(f'Test RMSE for CatBoost L2 is {np.sqrt(mean_squared_error(y_test, test_l2_cbr))}')